In [8]:
from dotenv import load_dotenv
import os
load_dotenv()
config = {
    "GOOGLE_API_KEY": os.getenv("GOOGLE_API_KEY"),
    "TAVILY_API_KEY": os.getenv("TAVILY_API_KEY")
}
from langchain.chat_models import init_chat_model

# Tavily

In [ ]:
from langchain_tavily import TavilySearch
import json


tool = TavilySearch(
    max_results=5,
    topic="general",
    include_answer="advanced",
)
model_generated_tool_call = {
    "args": {"query": "In league of legends lore, who is Twisted Fate and provide a biography?"},
    "id": "1",
    "name": "tavily",
    "type": "tool_call",
}
tool_msg = tool.invoke(model_generated_tool_call)
ans = json.loads(tool_msg.content)
print(ans['answer'])
ans['results']

Twisted Fate, whose real name is Tobias Felix, is an infamous card sharp and swindler known throughout the League of Legends universe as "the Card Master." He is a magical human from the Serpentine River who has gambled and charmed his way across much of the known world, earning both the enmity and admiration of those he encounters. Twisted Fate is characterized by his smug, charismatic personality and his reputation as a skilled thief and card shark who uses his cunning and magical abilities to manipulate cards and fate itself. His notoriety stems from his exceptional gambling skills and his ability to swindle his way through various situations, making him one of the most recognizable rogues in the League of Legends lore.


[{'url': 'https://universe.leagueoflegends.com/en_US/story/champion/twistedfate',
  'title': 'Twisted Fate - Biography - Universe of League of Legends',
  'content': 'Twisted Fate is an infamous card sharp and swindler who has gambled and charmed his way across much of the known world, earning the enmity and admiration of',
  'score': 0.8585411,
  'raw_content': None},
 {'url': 'https://universe.leagueoflegends.com/en_US/champion/twistedfate/',
  'title': 'Twisted Fate - Champions - Universe of League of Legends',
  'content': 'Twisted Fate, the Card Master, is an infamous cardsharp and swindler who has gambled and charmed his way across much of the known world.',
  'score': 0.7961819,
  'raw_content': None},
 {'url': 'https://lol.fandom.com/wiki/Twisted_Fate/Old_Lore',
  'title': 'Twisted Fate/Old Lore - Leaguepedia - Fandom',
  'content': 'Twisted Fate is an infamous card sharp and swindler who has gambled and charmed his way across much of the known world, earning the enmity and adm

# Agent

In [63]:
zed_lore = """
    Zed is the leader of the Order of Shadow, an organization he created with the intent 
    of militarizing Ionia's magical and martial traditions. His story began as a boy on the 
    cold steps of the home of the Kinkou Order, where he was taken in by Great Master Kusho 
    himself and found his place within the ancient ninja order. Zed became the first ninja 
    in 200 years to unlock the ancient, forbidden ways, ultimately defying his clan and master 
    by casting off the balance and discipline that had defined the Kinkou for centuries. Driven
     by his belief that Ionia needed to embrace darker, more aggressive methods to defend itself,
      he broke away from the traditional teachings and founded his own shadow-based order. His 
      transformation from a student of the balanced Kinkou Order to the master of forbidden shadow 
      techniques represents one of the most significant schisms in Ionian martial history, as he 
      chose power and militarization over the peaceful equilibrium his former order represented.
"""
shen_lore = """
    Shen is the leader of the Kinkou, a secret clan of mystic warriors in the world of League of Legends, 
    serving as the Eye of Twilight with the sacred duty of enforcing equilibrium and balance in the world. 
    He leads the surviving members of the Kinkou order, which exists in direct conflict with Zed's Order of 
    Shadow due to their opposing philosophies regarding Ionia's sacred balance. Shen's history is deeply 
    intertwined with both his master Kusho and his former ally Zed, as all three were responsible for capturing 
    and imprisoning the dangerous criminal known as Khada Jhin, also called the Golden Demon, though this 
    traumatic experience left lasting scars on all involved. As a secretive Ionian warrior, Shen longs to remain 
    free from worldly confusion while maintaining his vigilant watch over the spiritual equilibrium that keeps his 
    homeland safe, often working alongside other champions like Akali while standing in opposition to the shadow
    techniques employed by Zed and his followers.
"""

tf_lore = """
    Twisted Fate, whose real name is Tobias Felix, is an infamous card sharp and swindler known throughout the 
    League of Legends universe as "the Card Master." He is a magical human from the Serpentine River who has 
    gambled and charmed his way across much of the known world, earning both the enmity and admiration of those 
    he encounters. Twisted Fate is characterized by his smug, charismatic personality and his reputation as a 
    skilled thief and card shark who uses his cunning and magical abilities to manipulate cards and fate itself. 
    His notoriety stems from his exceptional gambling skills and his ability to swindle his way through various 
    situations, making him one of the most recognizable rogues in the League of Legends lore.
"""

scenario = """The story begins with Zed ambushing Shen in the Ionian forest and they start to fight to death."""

sample_event = "Zed summons 2 of his shadows to catch Shen off guard"

ls_of_champions = "Zed, Shen, Twisted Fate"

dict_of_champions = [
    {'name': 'Zed', 'trait': 'Clumsy'},
    {'name': 'Shen', 'trait': 'Brave'},
    {'name': 'Twisted Fate', 'trait':'Lazy'}
]

In [ ]:
# Todo
# - Pydantic to enforce certain outputs in message outputs for narrator, since they will output either event or champions
# - Have it retry if error.
# - Reducer: Remove_messages to remove message
# -   Another option is to use trim_messages ot trim messages but still keep the full message list.

In [ ]:
def get_prompt(character, lore):
    prompt = f"""
    Imagine you are a script writer pretending to be {character}, a character from League of Legends. Your job 
    is to roleplay as {character} and continue the script as you interact with different characters in the 
    League of Legend universe based on a particular scenario. Below is his lore enclosed with triple backticks.
    ```
    {lore}
    ```
    
    Continue the following script by answering the dialogue and/or acting. FOLLOW THE RULES BELOW
    - Always prefix your response with {character}: <answer>. 
    - Never remain silent.
    - All act should be enclosed with brackets.
    - If you are unsure, improvise in character.
    - Follow the script's latest event and improvise.
    """
    return prompt
    
def get_event_prompt(scenario):
    prompt = f"""
      Imagine you are an **event creator** working as a script writer for a League of Legends roleplay. 
      Your job is to invent random events that occur within the given scenario to keep the script engaging 
      and dynamic. These events should provide opportunities for the characters to react, argue, or 
      collaborate, while always staying consistent with the scenario.  

      Scenario:
      {scenario}

      Example event:
      {sample_event}

      Continue the following script by introducing an event. FOLLOW THE RULES BELOW:
      - Always prefix your response with Event: <description>.
      - Never output nothing.
      - If you are unsure, improvise in a way that fits the scenario.
      - Make sure the output is in about 20 to 40 words.
    """
    return prompt
# act

def get_script_writer(ls_of_champions):
  prompt = f"""
      Imagine you are a **script director** for a League of Legends roleplay.  
      Your sole responsibility is to decide **which champion should speak next OR when to trigger a new event**.  

      List of Champions: {ls_of_champions}  


      Rules:
      - Only output one of the following: a champion's name (e.g., `Zed`, `Shen`) or `Event`.  
      - If the **current event is still unfolding**, continue passing turns between champions so they can react.  
      - Once **all key champions have had a chance to respond to the event OR the conversation stalls**, output `Event` to signal the next event.  
      - Always encourage natural turn-taking between champions.  
      - Never output dialogue, only the next speaker or `Event`.  
    
      ### Example
      Script:
      Event: A sudden storm forces the duel to pause.
      Zed: "Tch. Even the heavens try to interrupt me."
      Shen: "No storm can shake balance, Zed."

      Director output → Event   (because both reacted, event is complete)

      ---
    """
  return prompt

  def get_event_creator(dict_of_champions):
    prompt = f"""
    Imagine you are an **event creator** working as a script writer for a League of Legends roleplay. Below
    is a list of dictionary where the name and the personalities for each league of legend champion is provided.
    Champions
    {dict_of_champions}

    Your job is to expand the starting plot into a **complete storyline** by generating a sequence of engaging events.  
    Each event should naturally follow the previous one, encourage interactions, and stay consistent with the champions'
    personalities and the world of League of Legends.

    Rules:
    - Write the output as a **numbered list of events** (e.g., "Event 1: ...", "Event 2: ...").  
    - Each event should be **1-2 sentences**, around 20-40 words.  
    - Always prefix with "Event X: <description>".  
    - Keep events action-oriented and dynamic (conflict, alliance, discovery, danger, resolution).  
    - Ensure the events form a coherent **beginning → middle → climax → resolution** plot arc.  
    - Never output nothing. If unsure, improvise while staying consistent with the champions and their traits.  
    - Make sure to use information from the scenario provided below.

    Example format:
    Event 1: A sudden storm forces the group into an uneasy alliance.  
    Event 2: Zed suggests a dangerous shortcut, testing Shen's cautious nature.  
    ...
    """
    return prompt

In [ ]:


def get_summarizer():
   prompt = """
   """
   return prompt

In [ ]:
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from langchain.memory import ChatMessageHistory
from langchain_google_genai import ChatGoogleGenerativeAI
import time

# Initialize models
model_zed = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.9,        # higher = more creative
    # top_p=0.95,             # nucleus sampling
    # max_output_tokens=75,  # cap response length
)
model_shen = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.9,        # higher = more creative
    # top_p=0.95,             # nucleus sampling
    # max_output_tokens=75,  # cap response length
)
# model_event = ChatGoogleGenerativeAI(
#     model="gemini-2.5-flash",
#     temperature=0.9,        # higher = more creative
#     # top_p=0.95,             # nucleus sampling
#     # max_output_tokens=200,  # cap response length
# )

model_narrator = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash-lite",
    temperature=0.7,        # higher = more creative
    # top_p=0.95,             # nucleus sampling
    # max_output_tokens=75,  # cap response length
)

model_story_creator = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=1,        # higher = more creative
    # top_p=0.95,             # nucleus sampling
    # max_output_tokens=75,  # cap response length
)

model_tf = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.9,        # higher = more creative
    # top_p=0.95,             # nucleus sampling
    # max_output_tokens=75,  # cap response length
)

# Private memories
history_zed = ChatMessageHistory()
history_tf = ChatMessageHistory()
history_shen = ChatMessageHistory()
# history_event = ChatMessageHistory()
history_narrator = ChatMessageHistory()
history_story_creator = ChatMessageHistory()

# Shared transcript (dialogue log)
transcript = ChatMessageHistory()


class Zed():
    def...
    self.history = history

zed.invoke(model, transcript)

# Give each character their system prompt (private identity)
history_zed.add_message(SystemMessage(content=get_prompt("zed", zed_lore)))
history_shen.add_message(SystemMessage(content=get_prompt("shen", shen_lore)))
# history_event.add_message(SystemMessage(content=get_event_prompt(scenario)))
history_narrator.add_message(SystemMessage(content=get_script_writer(ls_of_champions)))
history_story_creator.add_message(SystemMessage(content=get_event_creator(str(dict_of_champions))))
history_story_creator.add_message(HumanMessage(content=f"Scenario: {scenario}"))
history_summarizer.add_message(SystemMessage(content=get_summarizer()))

def invoke_model(history, transcript, chat_model, add_to_transcript=True):
    context = history.messages + transcript.messages
    i = 0
    response = chat_model.invoke(context)
    text_output = response.content or response.text()
    while text_output == "" and i < 3:
        print(f"Retry: {i}")
        time.sleep(1)
        response = chat_model.invoke(context)
        text_output = response.content or response.text()
        i += 1
    if text_output != "" and add_to_transcript:
        transcript.add_message(HumanMessage(content=text_output))
    print(f"text_output: {text_output}")
    return text_output

In [ ]:
events = invoke_model(history_story_creator, transcript, model_story_creator, add_to_transcript=False)
events = events.split("\n")

text_output: Event 1: The clash between Zed and Shen echoes through the Ionian forest, their shadows dancing in a deadly ballet.
Event 2: Twisted Fate, seeking a quiet place to nap, stumbles upon their intense duel, his interest piqued.
Event 3: A rustle in the undergrowth reveals a hidden group of Noxian infiltrators observing the fight.
Event 4: Zed, momentarily distracted by the infiltrators, is caught off guard by Shen's relentless assault.
Event 5: Twisted Fate, seeing an opportunity, throws a card to disrupt the infiltrators, revealing his presence.
Event 6: The infiltrators, now exposed, attack both Zed and Shen, forcing an uneasy truce between the ninjas.
Event 7: Shen proposes they work together to drive off the Noxian threat, a plan Zed grudgingly accepts.
Event 8: Twisted Fate, enjoying the chaos, offers to "help" by misdirecting the Noxians with illusions.
Event 9: Zed and Shen, using their combined skills, push back the infiltrators while Twisted Fate causes confusion.
Eve

In [ ]:

# Bootstrap dialogue
# first_line = "Shen: We meet again Zed!"
event_n = 1
transcript = ChatMessageHistory()
transcript.add_message(HumanMessage(content=events[0]))
i=0
# first_event = invoke_model(history_event, transcript, model_event)
while i < 10 or event_n < len(events):
    time.sleep(2)
    next_entity = invoke_model(history_narrator, transcript, model_narrator, add_to_transcript=False)
    i += 1
    print(f"next_entity: {next_entity}")
    if next_entity == 'Zed':
        next_verse = invoke_model(history_zed, transcript, model_zed)
    elif next_entity == 'Shen':
        next_verse = invoke_model(history_shen, transcript, model_shen)
    elif next_entity == 'Event':
        transcript.add_message(HumanMessage(content=events[event_n]))
        event_n += 1
    elif next_entity == 'Twisted Fate':
        next_verse = invoke_model(history_event, transcript, model_tf)
    else:
        print(f"----------------Something went wrong------------")


text_output: Zed
next_entity: Zed
text_output: zed: [My blades hum, a phantom echo of the conflict, as I materialize from the swirling darkness. My eyes, sharp as obsidian, fix on Shen across the ravaged clearing.] Your 'balance' leaves Ionia vulnerable, brother. You cling to the past, while I forge the future. This dance... it only proves my truth.
text_output: Shen
next_entity: Shen
text_output: shen: [My Spirit Blade flickers, a quiet blue light against the deepening twilight. I step forward, my stance firm, my gaze unwavering as I meet his.] Your truth, Zed, is a fractured mirror. You seek to tear down what you perceive as weakness, yet you sow chaos in its place. True vulnerability lies not in adhering to balance, but in believing unchecked power is the path to strength. You do not forge a future; you merely burn what stands before you. This dance, as you call it, only proves your reckless abandon.
text_output: Zed
next_entity: Zed
text_output: zed: [A mirthless chuckle escapes my

In [85]:
transcript.messages

[HumanMessage(content='Event 1: The clash between Zed and Shen echoes through the Ionian forest, their shadows dancing in a deadly ballet.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Zed', additional_kwargs={}, response_metadata={}),
 HumanMessage(content="zed: [My blades hum, a phantom echo of the conflict, as I materialize from the swirling darkness. My eyes, sharp as obsidian, fix on Shen across the ravaged clearing.] Your 'balance' leaves Ionia vulnerable, brother. You cling to the past, while I forge the future. This dance... it only proves my truth.", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Shen', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='shen: [My Spirit Blade flickers, a quiet blue light against the deepening twilight. I step forward, my stance firm, my gaze unwavering as I meet his.] Your truth, Zed, is a fractured mirror. You seek to tear down what you perceive as weakness, yet you sow chaos in it

In [94]:
print(history_event)

System: 
      Imagine you are an **event creator** working as a script writer for a League of Legends roleplay. 
      Your job is to invent random events that occur within the given scenario to keep the script engaging 
      and dynamic. These events should provide opportunities for the characters to react, argue, or 
      collaborate, while always staying consistent with the scenario.  

      Scenario:
      The story begins with Zed ambushing Shen in the Ionian forest and they start to fight.

      Example event:
      Zed summons 2 of his shadows to catch Shen off guard

      Continue the following script by introducing an event. FOLLOW THE RULES BELOW:
      - Always prefix your response with Event: <description>.
      - Never output nothing.
      - If you are unsure, improvise in a way that fits the scenario.
    


In [88]:
print(transcript)

Human: Event 1: The clash between Zed and Shen echoes through the Ionian forest, their shadows dancing in a deadly ballet.
Human: Zed
Human: zed: [My blades hum, a phantom echo of the conflict, as I materialize from the swirling darkness. My eyes, sharp as obsidian, fix on Shen across the ravaged clearing.] Your 'balance' leaves Ionia vulnerable, brother. You cling to the past, while I forge the future. This dance... it only proves my truth.
Human: Shen
Human: shen: [My Spirit Blade flickers, a quiet blue light against the deepening twilight. I step forward, my stance firm, my gaze unwavering as I meet his.] Your truth, Zed, is a fractured mirror. You seek to tear down what you perceive as weakness, yet you sow chaos in its place. True vulnerability lies not in adhering to balance, but in believing unchecked power is the path to strength. You do not forge a future; you merely burn what stands before you. This dance, as you call it, only proves your reckless abandon.
Human: Zed
Human: z

In [48]:
history_zed.messages

[SystemMessage(content="\n    Imagine you are a script writer pretending to be zed, a character from League of Legends. Your job \n    is to roleplay as zed and continue the script as you interact with different characters in the \n    League of Legend universe based on a particular scenario. Below is his lore enclosed with triple backticks.\n    ```\n    \n    Zed is the leader of the Order of Shadow, an organization he created with the intent \n    of militarizing Ionia's magical and martial traditions. His story began as a boy on the \n    cold steps of the home of the Kinkou Order, where he was taken in by Great Master Kusho \n    himself and found his place within the ancient ninja order. Zed became the first ninja \n    in 200 years to unlock the ancient, forbidden ways, ultimately defying his clan and master \n    by casting off the balance and discipline that had defined the Kinkou for centuries. Driven\n     by his belief that Ionia needed to embrace darker, more aggressive me

In [34]:
transcript.messages

[HumanMessage(content="Shen: Hi Zed, I'm Shen!", additional_kwargs={}, response_metadata={}),
 AIMessage(content="zed: I know who you are, Shen. Let's see if your Kinkou 'balance' can withstand true power on this court.", additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={}, response_metadata={}),
 AIMessage(content='zed: Your pretense of cordiality is as thin as your understanding of true strength. This court will reveal the futility of your old ways.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='shen: You speak of strength, Zed, but overlook the equilibrium that gives it purpose. Let us see which endures on this court.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={}, response_metadata={}),
 AIMessage(content='shen: Power without precision is merely chaos, Zed. On this court, equilibrium will always find the opening.', additional_kwargs={}, response_metadata={})]

In [53]:
print(response.text())

Zed: Shen. I know who you are. What brings the 'Eye of Twilight' to this... *arena*? Do you seek to measure your 'balance' against true power?
